# Extract, Transform, Load process for small molecule database

### Step 1: Extract the data from PubChem.

To narrow down the amount of molecules, five classes of drugs which are targeting the nervous system were selected.

On https://pubchem.ncbi.nlm.nih.gov/ five queries were used: Benzodiazepines, Antidepressants, Antiepileptics, Antipsychotics, Stimulants. For all of these, a CSV was exported containing the search results with several columns of information.

In [ ]:
import pandas as pd
import mysql.connector
import requests
from time import sleep
import os
import csv
import time
from pathlib import Path
import mysql.connector
import math

To add a bit more information, the "requests" library was used. The csv files were enriched with additional data (Mechanism of action and main medical use).

In [ ]:
# Input CSV files to process (one enriched output per file)
FILES = [
    "PubChem_compound_text_Stimulants.csv",
    "PubChem_compound_text_Antidepressants.csv",
    "PubChem_compound_text_Antiepileptics.csv",
    "PubChem_compound_text_Antipsychotics.csv",
    "PubChem_compound_text_Benzodiazepines_APPROVED.csv",
]

# Base directory containing all CSV files
BASE_DIR = Path("nervous_system_drugs")

# Column holding PubChem Compound IDs
CID_COLUMN = "Compound_CID"

# Optional processing limit (None = process all rows)
MAX_ROWS = None

# Delay between API requests to avoid rate-limiting
SLEEP_TIME = 0.2


def get_pubchem_text(cid):
    """Fetch mechanism of action and main medical use from PubChem."""
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON/"

    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        data = r.json()
    except Exception as e:
        print(f"[!] Error fetching/parsing CID {cid}: {e}")
        return None, None

    mech = None
    use = None

    # Recursively traverse PubChem's nested section structure
    def walk_sections(sections):
        nonlocal mech, use
        for sec in sections:
            heading = sec.get("TOCHeading", "").lower()

            for info in sec.get("Information", []):
                text_chunks = info.get("Value", {}).get("StringWithMarkup", [])
                if not text_chunks:
                    continue

                text = text_chunks[0].get("String", "").strip()
                if not text:
                    continue

                # Heuristic matching based on section headings
                if ("mechanism of action" in heading or
                    ("pharmacology" in heading and mech is None)):
                    mech = mech or text

                if ("indication" in heading or
                    "drug indication" in heading or
                    "indications and usage" in heading or
                    ("therapeutic uses" in heading and use is None) or
                    ("clinical information" in heading and use is None)):
                    use = use or text

            # Recurse into subsections
            if "Section" in sec:
                walk_sections(sec["Section"])

    try:
        walk_sections(data["Record"]["Section"])
    except KeyError:
        # Some CIDs do not follow the standard PubChem structure
        pass

    return mech, use


def enrich_csv(input_csv: Path):
    # Output file name: *_ENRICHED.csv
    output_csv = input_csv.with_name(input_csv.stem + "_ENRICHED.csv")
    print(f"\n=== Processing {input_csv.name} ===")

    df = pd.read_csv(input_csv)

    # Columns to be populated from PubChem
    df["Mechanism_of_Action"] = None
    df["Main_Medical_Use"] = None

    # Cache avoids repeated API calls for duplicate CIDs
    cache = {}

    for i, row in df.iterrows():
        if MAX_ROWS and i >= MAX_ROWS:
            print(f"Reached limit of {MAX_ROWS} rows.")
            break

        cid = row.get(CID_COLUMN)
        if pd.isna(cid):
            continue

        if cid not in cache:
            print(f"CID {cid} ({i+1})")
            cache[cid] = get_pubchem_text(cid)
            sleep(SLEEP_TIME)

        df.at[i, "Mechanism_of_Action"], df.at[i, "Main_Medical_Use"] = cache[cid]

    df.iloc[:MAX_ROWS].to_csv(output_csv, index=False)
    print(f"Saved → {output_csv.name}")


def main():
    # Process all configured CSV files
    for fname in FILES:
        path = BASE_DIR / fname
        if path.exists():
            enrich_csv(path)
        else:
            print(f"[!] File not found: {path}")


if __name__ == "__main__":
    main()

Look at one example:

In [37]:
benzodiazepines_enriched = pd.read_csv(r"nervous_system_drugs/PubChem_compound_text_Benzodiazepines_ENRICHED.csv")

benzodiazepines_enriched.iloc[2, :]

Compound_CID                                                                       2118
Name                                                                         Alprazolam
Synonyms                              alprazolam|Xanax|28981-97-7|Niravam|Solanax|Tr...
Molecular_Weight                                                                  308.8
Molecular_Formula                                                            C17H13ClN4
Polar_Area                                                                         43.1
Complexity                                                                        434.0
XLogP                                                                               2.1
Heavy_Atom_Count                                                                     22
H-Bond_Donor_Count                                                                    0
H-Bond_Acceptor_Count                                                                 3
Rotatable_Bond_Count            

Add a new column to all of the csv files which labels the class of the molecule

In [ ]:
# Map each enriched CSV file to its corresponding drug class label
FILES_AND_CLASSES = {
    "nervous_system_drugs/PubChem_compound_text_Antidepressants_ENRICHED.csv": "antidepressant",
    "nervous_system_drugs/PubChem_compound_text_Antiepileptics_ENRICHED.csv": "antiepileptic",
    "nervous_system_drugs/PubChem_compound_text_Antipsychotics_ENRICHED.csv": "antipsychotic",
    "nervous_system_drugs/PubChem_compound_text_Benzodiazepines_ENRICHED.csv": "benzodiazepine",
    "nervous_system_drugs/PubChem_compound_text_Stimulants_ENRICHED.csv": "stimulant",
}

# Iterate over all files and apply the corresponding drug class
for filename, drug_class in FILES_AND_CLASSES.items():
    if not os.path.exists(filename):
        print(f"[!] File not found: {filename}")
        continue

    print(f"Processing: {filename}")

    df = pd.read_csv(filename)

    # Add or overwrite a uniform class label for all rows
    df["drug_class"] = drug_class

    # Write changes back to the same file
    df.to_csv(filename, index=False)

    print(f"✅ Added drug_class = '{drug_class}' to {filename}")

print("\nAll files processed.")

Processing: nervous_system_drugs/PubChem_compound_text_Benzodiazepines_ENRICHED.csv
✅ Added drug_class = 'benzodiazepine' to nervous_system_drugs/PubChem_compound_text_Benzodiazepines_ENRICHED.csv

🎉 All files processed.


Also, the benzodiazepine dataset had to be filtered, because it was pretty big with 300'000 entries (containing not only "real" benzodiazepines but also structurally related compounds).
For this, a list containing the approved benzodiazepines was used.

In [ ]:
INPUT_CSV = r"nervous_system_drugs/PubChem_compound_text_Benzodiazepines.csv"
OUTPUT_CSV = r"nervous_system_drugs/PubChem_compound_text_Benzodiazepines_APPROVED.csv"

# List of commonly used / approved benzodiazepines (EU/US, etc.)
APPROVED_BENZOS = {
    # ===== CORE GLOBAL BENZODIAZEPINES =====
    "alprazolam",
    "bromazepam",
    "chlordiazepoxide",
    "clobazam",
    "clonazepam",
    "clorazepate",
    "diazepam",
    "estazolam",
    "flunitrazepam",
    "flurazepam",
    "halazepam",
    "lorazepam",
    "lormetazepam",
    "midazolam",
    "nitrazepam",
    "oxazepam",
    "prazepam",
    "quazepam",
    "temazepam",
    "triazolam",

    # ===== ACTIVE METABOLITES / PRODRUGS =====
    "nordazepam",
    "desmethyldiazepam",
    "desalkylflurazepam",
    "avizafone",
    "rilmazafone",
    "fosazepam",

    # ===== THIENODIAZEPINES (REAL MEDICAL USE) =====
    "etizolam",
    "brotizolam",
    "clotiazepam",

    # ===== REGIONAL / JAPAN / EU ONLY =====
    "mexazolam",
    "pinazepam",
    "ketazolam",
    "loprazolam",
    "cinazepam",
    "medazepam",
    "camazepam",
    "iprazepam",
    "metaclazepam",
    "nimetazepam",
    "pivoxazepam",
    "bentazepam",

    # ===== DISCONTINUED BUT HISTORICALLY PRESCRIBED =====
    "arfenda",
    "fletazepam",
    "flutoprazepam",
    "meclonazepam",
    "tetrazepam",
    "safrazepam",
    "suriclone",

    # ===== RARE / HOSPITAL-ONLY / LEGACY =====
    "remimazolam",
    "tolazepam",
    "ro09-9212",
    "ro15-1788",       # flumazenil (antidote, still a benzodiazepine core)
    "flutazolam",
    "adaline",
    "devazepide",

    # ===== INACTIVE / STRUCTURAL BENZOS WITH MEDICAL MENTIONS =====
    "bifemelane",
    "pazinaclone",
    "zometapine"
}




def normalize_name(name: str) -> str:
    """
    Normalize a compound name for comparison:
    - lowercase
    - strip whitespace
    - cut off anything after '[' or '(' (e.g. 'Diazepam [INN]' -> 'diazepam')
    """
    if pd.isna(name):
        return ""
    name = str(name).lower().strip()
    for sep in ["[", "("]:
        if sep in name:
            name = name.split(sep)[0]
    return name.strip()


def main():
    if not os.path.exists(INPUT_CSV):
        raise FileNotFoundError(f"Input file not found: {INPUT_CSV}")

    df = pd.read_csv(INPUT_CSV)

    if "Name" not in df.columns:
        raise KeyError("Column 'Name' not found in CSV")

    # Create normalized name column
    df["name_norm"] = df["Name"].apply(normalize_name)

    # Filter to only rows whose normalized name is in our approved list
    mask = df["name_norm"].isin(APPROVED_BENZOS)
    df_approved = df[mask].copy()

    # Drop helper column before saving
    df_approved.drop(columns=["name_norm"], inplace=True)

    print(f"Total rows in original file: {len(df)}")
    print(f"Rows matching approved benzodiazepines: {len(df_approved)}")

    df_approved.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved approved benzodiazepines to: {OUTPUT_CSV}")


if __name__ == "__main__":
    main()


C:\Users\robin\AppData\Local\Temp\ipykernel_22292\1184772841.py:106: DtypeWarning: Columns (2,30,31,33,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_CSV)


Total rows in original file: 292056
Rows matching approved benzodiazepines: 40
Saved approved benzodiazepines to: nervous_system_drugs/PubChem_compound_text_Benzodiazepines_APPROVED.csv


### Step 2: Create a fitting database which contains the same columns in a table like in the csv files.

```

CREATE DATABASE IF NOT EXISTS chemdb;
USE chemdb;

```

```

CREATE TABLE nervous_system_drugs (
    id                              INT AUTO_INCREMENT PRIMARY KEY,
    Compound_CID                    INT,
    Name                            VARCHAR(255),
    Synonyms                        TEXT,
    Molecular_Weight                DECIMAL(12,4),
    Molecular_Formula               VARCHAR(50),
    Polar_Area                      DECIMAL(12,4),
    Complexity                      DECIMAL(12,4),
    XLogP                           DECIMAL(12,4),
    Heavy_Atom_Count                INT,
    `H-Bond_Donor_Count`            INT,
    `H-Bond_Acceptor_Count`         INT,
    Rotatable_Bond_Count            INT,
    InChI                           TEXT,
    SMILES                          TEXT,
    InChIKey                        VARCHAR(255),
    IUPAC_Name                      TEXT,
    Exact_Mass                      DECIMAL(12,6),
    Monoisotopic_Mass               DECIMAL(12,6),
    Charge                          INT,
    Covalent_Unit_Count             INT,
    Isotopic_Atom_Count             INT,
    Total_Atom_Stereo_Count         INT,
    Defined_Atom_Stereo_Count       INT,
    Undefined_Atom_Stereo_Count     INT,
    Total_Bond_Stereo_Count         INT,
    Defined_Bond_Stereo_Count       INT,
    Undefined_Bond_Stereo_Count     INT,
    Linked_PubChem_Literature_Count INT,
    Linked_PubChem_Patent_Count     INT,
    Linked_PubChem_Patent_Family_Count INT,
    MeSH_Headings                   TEXT,
    Annotation_Content              TEXT,
    Annotation_Type_Count           INT,
    Linked_BioAssays                TEXT,
    Create_Date                     VARCHAR(20),   -- or DATE if you later convert YYYYMMDD
    Data_Source                     TEXT,
    Data_Source_Category            TEXT,
    Tagged_by_PubChem               TEXT,
    Mechanism_of_Action             TEXT,
    Main_Medical_Use                TEXT,
    drug_class                      VARCHAR(50)    -- 👈 added for your 5 groups
);

-- Optional indexes (nice for performance)
CREATE INDEX idx_cid ON nervous_system_drugs (Compound_CID);
CREATE INDEX idx_drug_class ON nervous_system_drugs (drug_class);



```

### Step 3: Import the csv files into the chemdb database using the import wizard in mysql workbench.

Somehow it didnt work with the benzodiazepine dataset. As an alternative way, the code below was used (with mysql connector)

In [ ]:
# Path to the enriched CSV file to be imported
CSV_FILE = r"C:/Users/robin/Dropbox/Robin_Desktop/3_Semester/Databases/ETL_Life_Science_Databases/nervous_system_drugs/Import_with_wizard_to_database/PubChem_compound_text_Benzodiazepines_ENRICHED.csv"

# Connect to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Labor.2024",
    database="chemdb"
)

cursor = conn.cursor()

# Load the CSV into a DataFrame
df = pd.read_csv(CSV_FILE)

# Parameterized INSERT statement matching the table schema
insert_sql = """
INSERT INTO nervous_system_drugs (
    Compound_CID, Name, Synonyms, Molecular_Weight, Molecular_Formula,
    Polar_Area, Complexity, XLogP, Heavy_Atom_Count,
    `H-Bond_Donor_Count`, `H-Bond_Acceptor_Count`, Rotatable_Bond_Count,
    InChI, SMILES, InChIKey, IUPAC_Name, Exact_Mass, Monoisotopic_Mass,
    Charge, Covalent_Unit_Count, Isotopic_Atom_Count,
    Total_Atom_Stereo_Count, Defined_Atom_Stereo_Count,
    Undefined_Atom_Stereo_Count, Total_Bond_Stereo_Count,
    Defined_Bond_Stereo_Count, Undefined_Bond_Stereo_Count,
    Linked_PubChem_Literature_Count, Linked_PubChem_Patent_Count,
    Linked_PubChem_Patent_Family_Count, MeSH_Headings,
    Annotation_Content, Annotation_Type_Count, Linked_BioAssays,
    Create_Date, Data_Source, Data_Source_Category, Tagged_by_PubChem,
    Mechanism_of_Action, Main_Medical_Use, drug_class
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

# Insert each row, converting pandas NaN values to SQL NULL
for _, row in df.iterrows():
    values = []
    for v in row:
        if pd.isna(v) or (isinstance(v, float) and math.isnan(v)):
            values.append(None)
        else:
            values.append(v)

    cursor.execute(insert_sql, tuple(values))

# Persist changes and cleanly close the connection
conn.commit()
cursor.close()
conn.close()

print("Benzodiazepines imported successfully (NaN → NULL).")

✅ Benzodiazepines imported successfully (NaN → NULL).


### Step 4: Get additional synonyms from other sources (Chembl and Wikidata)

Because this takes long to run, only every 10th entry from the files is used.

In [ ]:
# Input CSV files containing compounds to enrich with synonyms
CSV_FILES = [
    "PubChem_compound_text_Antidepressants_ENRICHED.csv",
    "PubChem_compound_text_Antiepileptics_ENRICHED.csv",
    "PubChem_compound_text_Antipsychotics_ENRICHED.csv",
    "PubChem_compound_text_Benzodiazepines_ENRICHED.csv",
    "PubChem_compound_text_Stimulants_ENRICHED.csv"
]

# Column used to link compounds across databases
INCHIKEY_COLUMN = "InChIKey"

# Output file collecting all retrieved synonyms
OUTPUT_CSV = "synonyms_from_chembl_wikidata.csv"

# Optional delay to avoid stressing external APIs
SLEEP_BETWEEN_REQUESTS = 0


# API endpoints
CHEMBL_BASE = "https://www.ebi.ac.uk/chembl/api/data/molecule/{inchikey}.json"
WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"


def load_inchikeys_from_csv(files, inchikey_col):
    """
    Load unique InChIKeys from multiple CSV files.
    Only every 10th row per file is considered to reduce runtime.
    """
    inchikeys = set()

    for filename in files:
        path = Path(filename)
        if not path.exists():
            print(f"Warning: File {filename} not found – skipping.")
            continue

        with open(path, newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)

            if inchikey_col not in reader.fieldnames:
                print(
                    f"Warning: Column '{inchikey_col}' not found in {filename}. "
                    f"Found columns: {reader.fieldnames}"
                )
                continue

            row_counter = 0
            for row in reader:
                row_counter += 1

                # Take only every 10th entry to limit API calls
                if row_counter % 10 != 0:
                    continue

                ik = (row.get(inchikey_col) or "").strip()
                if ik:
                    inchikeys.add(ik)

    print(f"Loaded {len(inchikeys)} unique InChIKeys from CSV files.")
    return sorted(inchikeys)


def get_chembl_synonyms_for_inchikey(inchikey: str):
    """Fetch synonyms for a compound from ChEMBL using its InChIKey."""
    url = CHEMBL_BASE.format(inchikey=inchikey)

    try:
        r = requests.get(url, timeout=15)
    except Exception as e:
        print(f"  [ChEMBL] Network error for {inchikey}: {e}")
        return []

    # 404 means no ChEMBL entry for this InChIKey (normal case)
    if r.status_code != 200:
        if r.status_code != 404:
            print(f"  [ChEMBL] HTTP {r.status_code} for {inchikey}")
        return []

    try:
        data = r.json()
    except Exception as e:
        print(f"  [ChEMBL] JSON parsing error for {inchikey}: {e}")
        return []

    syns = []
    for entry in data.get("molecule_synonyms", []):
        s = entry.get("synonyms")
        if s:
            syns.append(s.strip())

    return syns


def get_wikidata_synonyms_for_inchikey(inchikey: str):
    """
    Fetch synonyms from Wikidata via SPARQL using InChIKey (P235).
    Retrieves both the main label and alternative labels.
    """
    query = f"""
    SELECT ?itemLabel ?altLabel WHERE {{
      ?item wdt:P235 "{inchikey}".
      OPTIONAL {{ ?item skos:altLabel ?altLabel. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,de". }}
    }}
    """

    headers = {
        "Accept": "application/sparql-results+json",
        # Required by Wikidata for polite API usage
        "User-Agent": "SynonymFetcher/1.0 (contact: your_email@example.com)"
    }

    try:
        r = requests.get(
            WIKIDATA_SPARQL,
            params={"query": query},
            headers=headers,
            timeout=20
        )
    except Exception as e:
        print(f"  [Wikidata] Network error for {inchikey}: {e}")
        return []

    if r.status_code != 200:
        print(f"  [Wikidata] HTTP {r.status_code} for {inchikey}")
        return []

    try:
        data = r.json()
    except Exception as e:
        print(f"  [Wikidata] JSON parsing error for {inchikey}: {e}")
        return []

    syns = set()
    for row in data.get("results", {}).get("bindings", []):
        label = row.get("itemLabel", {}).get("value")
        alt = row.get("altLabel", {}).get("value")

        if label:
            syns.add(label.strip())
        if alt:
            syns.add(alt.strip())

    return sorted(syns)


def main():
    # Collect InChIKeys across all input files
    inchikeys = load_inchikeys_from_csv(CSV_FILES, INCHIKEY_COLUMN)

    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f_out:
        writer = csv.writer(f_out)
        writer.writerow(["inchikey", "source", "synonym"])

        for idx, ik in enumerate(inchikeys, start=1):
            print(f"[{idx}/{len(inchikeys)}] Processing {ik} ...")

            # ChEMBL synonyms
            chembl_syns = get_chembl_synonyms_for_inchikey(ik)
            print(f"  ChEMBL: {len(chembl_syns)} synonyms")
            for s in chembl_syns:
                writer.writerow([ik, "ChEMBL", s])

            # Wikidata synonyms
            wikidata_syns = get_wikidata_synonyms_for_inchikey(ik)
            print(f"  Wikidata: {len(wikidata_syns)} synonyms")
            for s in wikidata_syns:
                writer.writerow([ik, "Wikidata", s])

            # Flush to disk to avoid data loss during long runs
            f_out.flush()

            time.sleep(SLEEP_BETWEEN_REQUESTS)

    print(f"Done! Synonyms saved to: {OUTPUT_CSV}")


if __name__ == "__main__":
    main()

### Step 5: Import the additional synonyms into the database after creating a new table

Create new synonyms table in MySQL Database:

```
CREATE TABLE synonyms (
    id INT AUTO_INCREMENT PRIMARY KEY,
    inchikey VARCHAR(27) NOT NULL,
    source VARCHAR(50) NOT NULL,
    synonym TEXT NOT NULL
);
```

Afterwards, load the new synonyms into the table using python:

In [ ]:
df = pd.read_csv("synonyms_from_chembl_wikidata_utf8.csv")

# Clean column names (remove whitespace and potential UTF-8 BOM)
df.columns = [c.strip().lstrip("\ufeff") for c in df.columns]

# Normalize InChIKey column name if needed
if "inchikey" not in df.columns and "InChIKey" in df.columns:
    df = df.rename(columns={"InChIKey": "inchikey"})

# Keep only required columns and drop incomplete rows
df = df[["inchikey", "source", "synonym"]].dropna()

# Convert DataFrame to a list of tuples for executemany()
rows = list(df.itertuples(index=False, name=None))

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Labor.2024",
    database="chemdb"
)
cur = conn.cursor()

cur.executemany(
    "INSERT INTO synonyms (inchikey, source, synonym) VALUES (%s, %s, %s)",
    rows
)

conn.commit()
cur.close()
conn.close()

print(f"Import completed. Rows attempted to insert: {len(rows)}")